<div align="center">
  <img src='https://i.ibb.co/d064HCdW/androidstudio-icon-md.png' height="75" alt="Android Studio Logo"/>
  
  [![GitHub](https://img.shields.io/badge/GitHub-100000?style=for-the-badge&logo=github&logoColor=white)](https://github.com/mrtear/Extension-Source-Builder)
</div>

🚨 Note:
- *Runtime Duration (FREE):*
   - TPU Runtime: ~3h 20m | ~47GB RAM, 24vCPU
   - CPU Runtime: ~85h 40m | ~12GB RAM, 2vCPU

- *Before Closing:*
   - Properly terminate session: `Runtime → Disconnect and delete runtime`

##### *Make sure to run each cell in order.*

In [ ]:
#@title 🔧 Install Dependencies & Packages
import os
import re
import urllib.request

# Suppress intermediate output and install essential packages
!apt-get update -qq > /dev/null 2>&1 && apt-get install -qq -y openjdk-17-jdk-headless android-sdk gradle > /dev/null 2>&1

# Set the main SDK directory
destination = "/lib/android-sdk"

# Fetch the latest command-line tools URL for Linux
url = "https://developer.android.com/studio"
html = urllib.request.urlopen(url).read().decode('utf-8')
match = re.search(r"https:\/\/dl\.google\.com\/android\/repository\/commandlinetools-linux-[0-9]*_latest\.zip", html)

if match:
    tools_download_url = match.group(0)

    # Download the command-line tools zip quietly
    !curl --location -o /content/android.zip {tools_download_url} > /dev/null 2>&1

    # Create & Unpack the zip into the temporary directory /content quietly
    !mkdir -p /content/android-temp
    !unzip -q /content/android.zip -d /content/android-temp

    # Create the necessary directory structure
    !mkdir -p "{destination}/cmdline-tools/latest"

    # Move the contents to the correct location, avoiding nested directories
    !mv /content/android-temp/cmdline-tools/* "{destination}/cmdline-tools/latest" > /dev/null 2>&1

    # Clean up the temporary files
    !rm -rf /content/android-temp /content/android.zip /usr/lib/android-sdk/build-tools/debian > /dev/null 2>&1

    # sdkmanager configuration
    !yes | /lib/android-sdk/cmdline-tools/latest/bin/sdkmanager --licenses > /dev/null 2>&1
    !/lib/android-sdk/cmdline-tools/latest/bin/sdkmanager "platforms;android-34" "build-tools;34.0.0" "platform-tools" > /dev/null 2>&1

    # Set environment variables
    os.environ['ANDROID_HOME'] = destination
    os.environ['PATH'] += f":{destination}/cmdline-tools/latest/bin"

    # Print completion message
    print("🛠️ You're good to go! Clone your repo next!")
else:
    print("Failed to fetch the tools_download_url")

In [ ]:
# @title <img src='https://i.ibb.co/rG4XCHfP/github-mark-white.png' height="20" /></a> Clone & Clean Repository
# @markdown ### 1. Enter GitHub repository details:
username = "" # @param {type:"string"}
project_name = "extensions-source" # @param {type:"string"}
# @markdown ### 2. (Optional) Enter branch name (leave empty for default/`main`):
branch_name = "" # @param {type:"string"}
# @markdown ---
# @markdown ### 🧹 **Optional Cleaning**
# @markdown To speed up builds, you can choose to keep only specific folders.
# @markdown **Keep ONLY specific language folder(s)?** (comma-separated, leave empty to keep all)
# @markdown **e.g., `all,en`**
languages = "" # @param {type:"string"}
# @markdown **Keep ONLY specific source(s)?** (comma-separated, requires a *single* language to be set above)
# @markdown **e.g., `hehescans,9scans`**
sources = "" # @param {type:"string"}

import os
import subprocess
import shutil
from IPython.display import clear_output

clear_output()

def clone_repository(username, project_name, branch):
    """Clones the specified repository."""
    if not username or not project_name:
        print("❌ Error: GitHub username and project name are required.")
        return False

    repo_url = f"https://github.com/{username}/{project_name}.git"
    target_dir = "/content/extensions-source"

    print("="*50)
    print("STEP 1: CLONING REPOSITORY")
    print("="*50)

    try:
        os.chdir("/content")  # safe directory before deletions

        if os.path.exists(target_dir):
            print("🧹 Found existing repository, cleaning up first...")
            shutil.rmtree(target_dir)

        print(f"📥 Cloning '{project_name}' from GitHub...")
        git_cmd = ["git", "clone", "--depth", "1"]
        if branch:
            git_cmd.extend(["-b", branch])
        git_cmd.extend([repo_url, target_dir])

        subprocess.run(git_cmd, check=True)

        # Set up necessary properties files inside the cloned repo
        os.chdir(target_dir)
        with open("local.properties", "w") as f:
            f.write("sdk.dir=/lib/android-sdk\n")
        with open("gradle.properties", "a") as f:
            f.write("\norg.gradle.jvmargs=-Xmx32g -Dfile.encoding=UTF-8\n")

        print("✅ Repository cloned and configured successfully!")
        return True

    except subprocess.CalledProcessError as e:
        print(f"❌ Error: Git clone failed.")
        print(f"   Please check the username, project name, and branch.")
        print(f"   stderr: {e.stderr}")
        return False
    except Exception as e:
        print(f"❌ An unexpected error occurred: {e}")
        return False

def clean_repository(lang_str, source_str):
    """Intelligently cleans the repository based on user selections."""
    src_path = "/content/extensions-source/src"
    if not os.path.exists(src_path):
        return

    print("\n" + "="*50)
    print("STEP 2: CLEANING REPOSITORY")
    print("="*50)

    selected_langs = {lang.strip() for lang in lang_str.split(',') if lang.strip()}
    selected_sources = {src.strip() for src in source_str.split(',') if src.strip()}

    # Case 1: No cleaning needed
    if not selected_langs and not selected_sources:
        print("✅ No specific cleaning requested. Kept all sources.")
        return

    all_langs = [d for d in os.listdir(src_path) if os.path.isdir(os.path.join(src_path, d))]

    # Case 2: Clean sources within a single specified language
    if selected_sources and len(selected_langs) == 1:
        kept_lang = list(selected_langs)[0]
        print(f"🧹 Keeping sources in language: '{kept_lang}'")

        # Delete other language folders
        for lang_folder in all_langs:
            if lang_folder != kept_lang:
                shutil.rmtree(os.path.join(src_path, lang_folder))

        # Clean within the selected language folder
        lang_folder_path = os.path.join(src_path, kept_lang)
        all_sources_in_lang = [d for d in os.listdir(lang_folder_path) if os.path.isdir(os.path.join(lang_folder_path, d))]

        for source_folder in all_sources_in_lang:
            if source_folder not in selected_sources:
                shutil.rmtree(os.path.join(lang_folder_path, source_folder))

        print(f"✅ Cleaning complete. Kept sources: {', '.join(sorted(selected_sources))}")
        return

    # Case 3: Keep entire language folder(s), no specific sources
    if selected_langs:
        print(f"🧹 Keeping language folder(s): {', '.join(sorted(selected_langs))}")
        for lang_folder in all_langs:
            if lang_folder not in selected_langs:
                shutil.rmtree(os.path.join(src_path, lang_folder))
        print("✅ Cleaning complete.")
        return

    # Case 4: Sources provided but language not specified correctly
    if selected_sources:
        print("⚠️ Warning: Sources were specified, but a single language was not selected.")
        print("   No source-specific cleaning was performed.")


# --- Main Execution ---
if clone_repository(username, project_name, branch_name):
    clean_repository(languages, sources)
    print("\n🎉 All operations completed successfully!")

In [ ]:
# @title <img src='https://i.ibb.co/Z6kXZ1mc/androidstudio-1024x1024.png' height="20" /></a> Build Extension(s)
# @markdown ### Select a source language:
lang = "" # @param ["", "all", "ar", "bg", "ca", "cs", "de", "en", "es", "fr", "id", "it", "ja", "ko", "pl", "pt", "ru", "th", "tr", "uk", "vi", "zh"]
# @markdown ### Enter source name(s) (comma-separated, e.g., `hehescans,9scans`):
sources_input = "" # @param {type:"string"}
# @markdown ### Show full build output?:
show_output = True # @param {type:"boolean"}

import subprocess
import os
import glob
from IPython.display import clear_output

clear_output()

# Ensure gradlew is executable
subprocess.run("chmod +x gradlew", shell=True)

src_base = f"/content/extensions-source/src/{lang}" if lang else "/content/extensions-source/src"

# Determine sources to build
if sources_input.strip():
    sources = [s.strip() for s in sources_input.split(",") if s.strip()]
else:
    # Build all sources in the selected language(s)
    sources = [d for d in os.listdir(src_base) if os.path.isdir(os.path.join(src_base, d))]

if not sources:
    print("❌ No sources found to build.")
else:
    total = len(sources)
    success_count = 0

    for idx, src_name in enumerate(sources, 1):
        print("\n" + "="*50)
        print(f"[{idx}/{total}] ⏳ Building: {src_name}")
        print("="*50)

        # Build command
        build_cmd = f"./gradlew :src:{lang}:{src_name}:assembleDebug --console=plain" if lang else f"./gradlew :src:{src_name}:assembleDebug --console=plain"
        result = subprocess.run(build_cmd, shell=True, capture_output=True, text=True)

        if result.returncode == 0:
            # Try to locate APKs
            apk_dir = os.path.join(src_base, src_name, "build/outputs/apk/debug")
            apk_files = glob.glob(os.path.join(apk_dir, "*.apk"))
            print("\n✅ Build Succeeded!")
            print(f"   - Source: {src_name}")
            if apk_files:
                print(f"   - APKs at: {apk_dir}")
                print(f"   - APK files: {', '.join([os.path.basename(a) for a in apk_files])}")
            else:
                print("   - No APKs found. (Check build.gradle or output path)")
            success_count += 1
        else:
            print("\n❌ Build Failed!")
            if show_output:
                print("\n--- Full Build Output ---")
                print(result.stdout)
                print(result.stderr)

    print("\n" + "="*50)
    print("🏁 Build Summary")
    print(f"   Processed {total} source(s): {success_count} successful, {total - success_count} failed.")
    print("="*50)

In [ ]:
# @title ### 📤 Upload APK(s) & get Download Link with QR Code
# @markdown ### Select a source language:
lang = "" # @param ["", "all", "ar", "bg", "ca", "cs", "de", "en", "es", "fr", "id", "it", "ja", "ko", "pl", "pt", "ru", "th", "tr", "uk", "vi", "zh"]
# @markdown ### Enter source name(s) (comma-separated, e.g., `hehescans,9scans`):
source = "" # @param {type:"string"}
# @markdown ### Select link expiration time:
upload_time = "1h" # @param ["1h", "12h", "24h", "72h"]

# Install necessary libraries
!pip install -q requests pyqrcode pypng > /dev/null 2>&1

import os
import glob
import requests
import pyqrcode
import re
from IPython.display import clear_output, display, Image

clear_output()

def upload_and_display_apks(lang_list, src, time_option="1h"):
    """Finds, uploads, and displays info for a single source across multiple languages."""
    success_count = 0
    for lang_item in lang_list:
        apk_dir_path = f"/content/extensions-source/src/{lang_item}/{src}/build/outputs/apk/debug/"
        apk_files = glob.glob(os.path.join(apk_dir_path, "*.apk"))

        if not apk_files:
            print(f"❌ No APK found for '{src}' in language '{lang_item}'.")
            continue

        for apk_path in apk_files:
            apk_filename = os.path.basename(apk_path)
            version_match = re.search(r'-v([\d.]+)-debug\.apk', apk_filename)
            version = version_match.group(1) if version_match else "Unknown"

            try:
                # Upload to Catbox/Litterbox
                with open(apk_path, 'rb') as apk_file:
                    files = {
                        'fileToUpload': (apk_filename, apk_file),
                        'time': (None, time_option),
                        'reqtype': (None, 'fileupload')
                    }
                    response = requests.post(
                        "https://litterbox.catbox.moe/resources/internals/api.php",
                        files=files
                    )
                    response.raise_for_status()
                    dl_url = response.text

                # Generate QR code
                qr = pyqrcode.create(dl_url)
                qr_filename = f"qr_{src}_{lang_item}.png"
                qr.png(qr_filename, scale=8)

                # Display
                print("\n" + "="*50)
                print(f"✅ Success! ({lang_item})")
                print(f"   - Name:    {src}")
                print(f"   - Version: {version}")
                print(f"   - Link:    {dl_url}")
                print(f"   - Expires: {time_option}")
                print("="*50)
                display(Image(qr_filename))
                success_count += 1

            except Exception as e:
                print(f"❌ Error uploading '{apk_filename}': {e}")

    return success_count

# Prepare list of languages
if lang.lower() == "all" or not lang:
    lang_list = [
        d for d in os.listdir("/content/extensions-source/src")
        if os.path.isdir(os.path.join("/content/extensions-source/src", d))
    ]
else:
    lang_list = [lang]

# Process sources
sources = [src.strip() for src in source.split(',') if src.strip()]

if not sources or not lang_list:
    print("❌ Please select a language and enter at least one source name.")
else:
    total_sources = len(sources)
    total_success = 0
    for idx, src_name in enumerate(sources, 1):
        print("\n" + "*"*50)
        print(f"[{idx}/{total_sources}] 📤 Processing: {src_name}")
        print("*"*50)
        total_success += upload_and_display_apks(lang_list, src_name, upload_time)

    print("\n" + "="*50)
    print("🏁 Upload Summary")
    print(f"   Processed {total_sources} source(s): {total_success} APK(s) successfully uploaded.")
    print("="*50)